In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

print(os.getcwd())

/Users/buldiga/Documents/ml/book-recsys/notebooks


In [3]:
df = pd.read_csv('../large_files/interactions.csv')
users = pd.read_csv('../large_files/users.csv')

In [4]:
df["start_date"] = pd.to_datetime(df["start_date"])
min_date = df["start_date"].max().normalize() - pd.DateOffset(days=30)

In [5]:
merged = df.merge(users, on='user_id')

In [70]:
df[df['progress'] < 50].shape, df[df['progress'] > 50].shape

((826154, 5), (727426, 5))

In [11]:
submission = pd.read_csv("../large_files/sample_submission.csv")
pred_pop = pd.DataFrame({"user_id": submission["Id"].unique()})

In [72]:
user_ids_pred = set(list(pred_pop['user_id']))

In [73]:
user_ids = set(list(users['user_id']))

In [74]:
len(user_ids_pred.difference(user_ids)), len(user_ids_pred.intersection(user_ids))

(547, 2527)

In [12]:
usr = pred_pop.merge(users, how='left', on='user_id')

In [47]:
groups = merged.loc[merged["start_date"] > min_date, ["item_id", 'age']].fillna('25_34').groupby('age')

In [59]:
rec = {}
for name, group in groups:
    rec[name] = group['item_id'].value_counts().head(10).index.values
print(rec)

{'18_24': array([ 80003, 283713, 184549, 385281, 287060, 267817, 276903, 168963,
       112869,  56877]), '25_34': array([ 80003, 357309, 385281, 184549, 276903,  56877, 283713, 287060,
       168963, 262464]), '35_44': array([184549, 283713, 357309, 385281, 168963,  80003, 242176, 276903,
       287060,  56877]), '45_54': array([283713, 184549, 143175, 168963, 276903,  55466, 357309, 242176,
       321351,  50718]), '55_64': array([283713, 184549, 143175,  55466, 168963, 276903, 160349,  51423,
       323949,  79499]), '65_inf': array([283713, 184549, 143175,  55466, 160349,  51423, 374648, 276903,
       178529, 168963])}


In [90]:
counts = users.groupby('age').count()['user_id']

In [93]:
normalized_df=counts/counts.sum()


In [94]:
normalized_df

age
18_24     0.207025
25_34     0.267414
35_44     0.228593
45_54     0.164933
55_64     0.086000
65_inf    0.046035
Name: user_id, dtype: float64

In [97]:
missing = merged['age'].isnull()


In [100]:
normalized_df.values

array([0.20702544, 0.26741424, 0.22859287, 0.16493253, 0.08600012,
       0.0460348 ])

In [101]:
merged.loc[missing,'age'] = np.random.choice(normalized_df.index, size=len(merged[missing]),p=normalized_df.values)


In [9]:
merged.groupby(['age', 'sex'])['user_id'].count()

age     sex
18_24   0.0    175837
        1.0     45606
25_34   0.0    253867
        1.0     69536
35_44   0.0    235058
        1.0     80031
45_54   0.0    176918
        1.0     73505
55_64   0.0     94163
        1.0     38411
65_inf  0.0     50492
        1.0     21276
Name: user_id, dtype: int64

In [7]:
merged.columns

Index(['user_id', 'item_id', 'progress', 'rating', 'start_date', 'age', 'sex'], dtype='object')

In [6]:
merged['sex'] = merged['sex'].fillna(0.0)
merged['age'] = merged['age'].fillna('25_34')

In [7]:
groups = (
            merged.loc[merged["start_date"] > min_date, ["item_id", "age", "sex"]]
            .groupby(["age", 'sex'])
            )

In [8]:
recs = {}
for name, group in groups:
    recs[name] = (group["item_id"].value_counts().head(100).index.values)
    

In [9]:
recs[('18_24', 0.0)]

array([ 80003, 283713, 287060, 385281, 267817, 184549, 262464,  56877,
       131612, 334106, 168963, 357309, 187325, 112869, 376516, 276903,
       290266, 390771, 143175, 373521,  55466, 359609,  79499, 321351,
       137878, 292395, 117325,   5408,  51423, 280876,  11316, 127019,
       305061, 313220, 310921, 313229, 242176, 148610, 320087,  33801,
       140100,  51835,  85931,  40953, 201080, 374648, 114173, 354482,
       322941, 264133, 229939, 347178,  93751,  69198, 352701, 313288,
       229030,  61100, 104445,  23427, 325439,  80576, 364570, 386701,
       308074, 353554, 199019, 283021, 268571, 327870,  41727, 116391,
       120645, 329185, 266210, 352049, 327042, 229111, 327411, 263141,
       280283, 134683, 354564, 225853,   5409, 196397, 342090, 109091,
        93398, 344047,  38089,  21313, 194169, 210614, 231623, 306759,
        85489, 273112, 344431, 114343])

In [10]:
merged

,user_id,item_id,progress,rating,start_date,age,sex
0,90133,82910,100,NaN,2018-01-01,55_64,0.0
1,90133,257548,100,NaN,2018-01-01,55_64,0.0
2,90133,88712,100,NaN,2018-01-01,55_64,0.0
3,90133,376450,100,NaN,2018-01-01,55_64,0.0
4,90133,85323,100,NaN,2018-01-01,55_64,0.0
...,...,...,...,...,...,...,...
1333558,69006,117325,100,NaN,2019-12-29,25_34,1.0
1333559,134233,283713,0,NaN,2019-12-29,18_24,0.0
1333560,102504,50124,52,NaN,2019-12-30,65_inf,1.0
1333561,156085,123247,100,0.0,2019-12-30,65_inf,0.0


In [14]:
usr["sex"] = usr["sex"].fillna(0.0)
usr["age"] = usr["age"].fillna("25_34")

In [40]:
usr_id = 10001
full_readed = set(merged[(merged['user_id'] == usr_id) & (merged['progress'] == 100)]['item_id'].values)

In [38]:
merged[(merged['user_id'] == usr_id) & (merged['progress'] == 100)]

,user_id,item_id,progress,rating,start_date,age,sex
1015852,10001,186002,100,5.0,2018-12-07,25_34,0.0


In [21]:
u = users[users['user_id'] == usr_id]

In [41]:
pred = set(recs[(u['age'].values[0], u['sex'].values[0])])

In [46]:
recs[(u['age'].values[0], u['sex'].values[0])]

array([ 80003,  56877, 357309, 262464, 184549, 168963, 276903, 385281,
       287060, 334106, 143175, 273112, 226603, 383870, 242176, 143687,
       283713, 112869,  85489, 302067, 323949, 229111, 352701, 354482,
         9590, 249083, 187325, 232060, 229939,  80398, 367354, 131612,
        80576, 390771, 352049, 169354, 313220, 117325, 295495, 280876,
       267817, 229030, 104445, 383831, 140510, 374648,  21313, 127019,
       345570, 331291,  49048,  99132,  93751, 292395,  51423,  76348,
       352500,  51835, 352868, 325439, 263485, 344047, 389882, 194169,
        30755,  10543,  76747, 283800, 116391, 211381, 334123, 329185,
        33767, 148860, 229524,  23427, 132859, 199827, 331330, 148370,
       105797, 377922, 148610,  40953,  22119,  91236, 129825, 360740,
       321351,  93398,  72285, 310921, 379918, 257026, 140100, 358608,
       246948, 291259, 198945, 212419])

In [43]:
list(pred.difference(full_readed))

[242176,
 385281,
 257026,
 168963,
 80398,
 379918,
 334106,
 331291,
 131612,
 129825,
 198945,
 30755,
 360740,
 267817,
 127019,
 292395,
 56877,
 226603,
 280876,
 334123,
 10543,
 352049,
 229939,
 93751,
 76348,
 99132,
 263485,
 325439,
 262464,
 283713,
 331330,
 377922,
 21313,
 140100,
 105797,
 295495,
 143175,
 143687,
 321351,
 117325,
 249083,
 287060,
 383831,
 72285,
 352868,
 91236,
 22119,
 323949,
 390771,
 9590,
 374648,
 194169,
 51835,
 232060,
 148860,
 383870,
 148610,
 80003,
 23427,
 313220,
 310921,
 169354,
 148370,
 199827,
 229524,
 283800,
 49048,
 246948,
 229030,
 116391,
 276903,
 354482,
 211381,
 367354,
 291259,
 357309,
 352701,
 187325,
 80576,
 212419,
 76747,
 358608,
 93398,
 273112,
 140510,
 51423,
 329185,
 345570,
 184549,
 112869,
 33767,
 344047,
 85489,
 302067,
 352500,
 229111,
 40953,
 389882,
 132859,
 104445]

In [44]:
merged

,user_id,item_id,progress,rating,start_date,age,sex
0,90133,82910,100,NaN,2018-01-01,55_64,0.0
1,90133,257548,100,NaN,2018-01-01,55_64,0.0
2,90133,88712,100,NaN,2018-01-01,55_64,0.0
3,90133,376450,100,NaN,2018-01-01,55_64,0.0
4,90133,85323,100,NaN,2018-01-01,55_64,0.0
...,...,...,...,...,...,...,...
1333558,69006,117325,100,NaN,2019-12-29,25_34,1.0
1333559,134233,283713,0,NaN,2019-12-29,18_24,0.0
1333560,102504,50124,52,NaN,2019-12-30,65_inf,1.0
1333561,156085,123247,100,0.0,2019-12-30,65_inf,0.0
